## Week 8 Class 2: IP - Facililty Location Example 7.3

# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Data](#2) <br>
3. [Original Model Definition](#3)<br>
4. [Original Model Solution](#4)<br>
5. [Additional Constraints (Logical/Disjunctive) Model Definition](#5)<br>
6. [Additional Constraints (Logical/Disjunctive) Model Solution](#6)<br>

## 1. Problem Statement<a id=1></a>

A Facility Location problem is a blend of a transportation model with fixed costs. That is, we have a problem of moving products from up to four cities (New York, Atlanta, Chicago and Los Angeles) to the distribution centers in the East, South, Midwest and West.

##### [Back to Top](#Top)

## 2. Data<a id=2></a>

In [1]:
import pandas as pd
import pyomo.environ as pe
import matplotlib.pyplot as plt

You will have to change the path (below) to point to the file on your computer!

In [3]:
raw_data = pd.read_excel('../Class-One/w08-c01-logical-part-2.xlsx', 
                         sheet_name = 'Example 7.3 - Location')
raw_data

,Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,To,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,East,South,Midwest,West,Capacity,Annual cost ($000s),NaN,NaN
2,From,New York,206,225,230,290,150,6000,NaN,NaN
3,NaN,Atlanta,225,206,221,270,150,5500,NaN,NaN
4,NaN,Chicago,230,221,208,262,150,5800,NaN,NaN
5,NaN,Los Angeles,290,270,262,215,150,6200,NaN,NaN
6,NaN,Demand,100,150,110,90,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Decisions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
DV_indexes = ['E', 'S', 'M', 'W']
coef = pd.DataFrame(raw_data.iloc[range(2, 6), range(2, 6)])
coef.index = ['N', 'A', 'C','L']
coef.columns = DV_indexes
coef

,E,S,M,W
N,206,225,230,290
A,225,206,221,270
C,230,221,208,262
L,290,270,262,215


In [5]:
capcost = raw_data.iloc[range(2, 6), [6, 7]]
capcost.columns = ['capacity', 'annualcost']
capcost.index = coef.index
capcost

,capacity,annualcost
N,150,6000
A,150,5500
C,150,5800
L,150,6200


In [6]:
demand = pd.DataFrame()
demand.loc['E', 'demand'] = 100
demand.loc['S', 'demand'] = 150
demand.loc['M', 'demand'] = 110
demand.loc['W', 'demand'] = 90
demand

,demand
E,100.0
S,150.0
M,110.0
W,90.0


##### [Back to Top](#Top)

### 3. Original Model Definition<a id=3></a>

In [7]:
model = pe.ConcreteModel()

## Define Decision Variables

Define your decision variables. __NOTE:__ In this first problem, we have two sets of decision variables in our model. The `x` variables defined the quantities being shipped between each potential origin and the destination (E, M, S, and W). The `y` variables are the individual binary variables that define whether or not we use a particular facility. The variable `y` is indexed by the facility cities.

In [8]:
model.N = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.A = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.C = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.L = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.y = pe.Var(coef.index, domain = pe.Binary)

for DV in model.component_objects(pe.Var):
    DV.pprint()

N : Size=4, Index=N_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      E :     0 :  None :  None : False :  True : NonNegativeReals
      M :     0 :  None :  None : False :  True : NonNegativeReals
      S :     0 :  None :  None : False :  True : NonNegativeReals
      W :     0 :  None :  None : False :  True : NonNegativeReals
A : Size=4, Index=A_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      E :     0 :  None :  None : False :  True : NonNegativeReals
      M :     0 :  None :  None : False :  True : NonNegativeReals
      S :     0 :  None :  None : False :  True : NonNegativeReals
      W :     0 :  None :  None : False :  True : NonNegativeReals
C : Size=4, Index=C_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      E :     0 :  None :  None : False :  True : NonNegativeReals
      M :     0 :  None :  None : False :  True : NonNegativeReals
      S :     0 :  None :  None : False :  True : NonNegativeReals
      W :     

### Define Objective Function

Define your model objective function. Note that this will be the total costs which is calculated from the fixed costs and variable sales costs. This objective function represents but `sumproduct` function calls in the Excel worksheet. 

In [9]:
#obj funct minimize cost * (N A C L) + annual cost * y
model.obj = pe.Objective(expr = sum(coef.loc['N', index]*model.N[index] for index in DV_indexes)
                                    + sum(coef.loc['A', index]*model.A[index] for index in DV_indexes)
                                    + sum(coef.loc['C', index]*model.C[index] for index in DV_indexes)
                                    + sum(coef.loc['L', index]*model.L[index] for index in DV_indexes)
                                    + sum(capcost.loc[index,'annualcost']*model.y[index] 
                                          for index in coef.index),
                         sense = pe.minimize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 206*N[E] + 225*N[S] + 230*N[M] + 290*N[W] + 225*A[E] + 206*A[S] + 221*A[M] + 270*A[W] + 230*C[E] + 221*C[S] + 208*C[M] + 262*C[W] + 290*L[E] + 270*L[S] + 262*L[M] + 215*L[W] + 6000*y[N] + 5500*y[A] + 5800*y[C] + 6200*y[L]


### Define constraints

In [10]:
#Demand Constraints
model.constE = pe.Constraint(expr=model.N['E'] + model.A['E']+model.C['E'] + model.L['E'] 
                             >= demand.loc['E','demand'])
model.constS = pe.Constraint(expr=model.N['S'] + model.A['S']+model.C['S'] + model.L['S']
                             >= demand.loc['S', 'demand'])
model.constM = pe.Constraint(expr=model.N['M'] + model.A['M']+model.C['M'] + model.L['M']
                             >= demand.loc['M', 'demand'])
model.constW = pe.Constraint(expr=model.N['W'] + model.A['W']+model.C['W'] + model.L['W']
                             >= demand.loc['W', 'demand'])

# Capacity and Linking Constraints DV <= M*y
model.constlinkN = pe.Constraint(expr=sum(model.N[index] for index in DV_indexes) 
                                 <= capcost.loc['N','capacity'] * model.y['N'])
model.constlinkA = pe.Constraint(expr=sum(model.A[index] for index in DV_indexes) 
                                 <= capcost.loc['A','capacity']* model.y['A'])
model.constlinkC = pe.Constraint(expr=sum(model.C[index] for index in DV_indexes) 
                                 <= capcost.loc['C','capacity'] * model.y['C'])
model.constlinkL = pe.Constraint(expr=sum(model.L[index] for index in DV_indexes) 
                                 <= capcost.loc['L','capacity'] * model.y['L'])

for con in model.component_objects(pe.Constraint):
    print(con,con.pprint())

constE : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None : 100.0 : N[E] + A[E] + C[E] + L[E] :  +Inf :   True
constE None
constS : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None : 150.0 : N[S] + A[S] + C[S] + L[S] :  +Inf :   True
constS None
constM : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None : 110.0 : N[M] + A[M] + C[M] + L[M] :  +Inf :   True
constM None
constW : Size=1, Index=None, Active=True
    Key  : Lower : Body                      : Upper : Active
    None :  90.0 : N[W] + A[W] + C[W] + L[W] :  +Inf :   True
constW None
constlinkN : Size=1, Index=None, Active=True
    Key  : Lower : Body                                 : Upper : Active
    None :  -Inf : N[E] + N[S] + N[M] + N[W] - 150*y[N] :   0.0 :   True
constlinkN None
constlinkA : Size=1, Index=None, Active=True
    Key  : Lower : Body                 

##### [Back to Top](#Top)

### 4. Original Model Solution<a id=4></a>

One thing that I always fail to mention is that you absolutely do not need to redefine the `opt` object each time you run `opt.solve` on a model. The `opt` definition will take care of specifying to use `glpk` throughout the entire python session.

In [11]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

ok optimal


### Optimal Objective Value

In [12]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

optimal objective value minimum cost = $115770.00


### Optimal Decision Variables

In [13]:
DV_solution = pd.DataFrame()
for index in DV_indexes:
    DV_solution.loc['N', index] = model.N[index].value
    DV_solution.loc['A', index] = model.A[index].value
    DV_solution.loc['C', index] = model.C[index].value
    DV_solution.loc['L', index] = model.L[index].value
DV_solution

,E,S,M,W
N,100.0,0.0,50.0,0.0
A,0.0,150.0,0.0,0.0
C,0.0,0.0,0.0,0.0
L,0.0,0.0,60.0,90.0


The following shows which facilities are being utilized.

In [14]:
for index in coef.index:
    print(f'y{index}:',model.y[index].value)

yN: 1.0
yA: 1.0
yC: 0.0
yL: 1.0


##### [Back to Top](#Top)

### 5. Additional Constraints (Logical/Disjunctive) Model Definition<a id=5></a>

In each subsequent problem, we will just add constraints to the problem. We don't need to redefine the model, objective function, etc. if they aren't changing (and they aren't here). We are just adding constraints to the problems and this will **might** require adding new variables and we have to in this case.

Let's look at the coefficients associated with our objective functions and constraint equations again. 

In [15]:
coef

,E,S,M,W
N,206,225,230,290
A,225,206,221,270
C,230,221,208,262
L,290,270,262,215


In [16]:
capcost

,capacity,annualcost
N,150,6000
A,150,5500
C,150,5800
L,150,6200


In [17]:
demand

,demand
E,100.0
S,150.0
M,110.0
W,90.0


### 5.2 Threshold Problem

#### Define Decision Variables

In [18]:
#BINARY Threshold NM > 60 or 0 Indicator
model.yNM60 = pe.Var(domain=pe.Binary)

In [19]:
#New constraints
big_M = 150
#Threshold NM >=60 or must be 0: NM >= 60 * yNM and NM <= 150 * yNM
model.constNM601 = pe.Constraint(expr = model.N['M'] >= 60 * model.yNM60)
model.constNM602 = pe.Constraint(expr = model.N['M'] <= 150 * model.yNM60)

In [20]:
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

ok optimal
optimal objective value minimum cost = $116270.00


In [21]:
DV_solution = pd.DataFrame()
for index in DV_indexes:
    DV_solution.loc['N', index] = model.N[index].value
    DV_solution.loc['A', index] = model.A[index].value
    DV_solution.loc['C', index] = model.C[index].value
    DV_solution.loc['L', index] = model.L[index].value
DV_solution

,E,S,M,W
N,100.0,50.0,0.0,0.0
A,0.0,100.0,50.0,0.0
C,0.0,0.0,0.0,0.0
L,0.0,0.0,60.0,90.0


In [22]:
for index in coef.index:
    print(f'y{index}:', model.y[index].value)

yN: 1.0
yA: 1.0
yC: 0.0
yL: 1.0


In [24]:
print(f'yNM60:', model.yNM60.value)

yNM60: 0.0


#### Define Decision Variables

Now we already have the DV for each path quantitiy `model.x` and the Fixed Cost binary indicator `y`. We need to add two more variables to the model. 

In [25]:
#BINARY Constraint LE >= 50
model.yLE50 = pe.Var(domain=pe.Binary)
#BINARY Constraint NE <= 75
model.yNE75 = pe.Var(domain=pe.Binary)

Now define the additional constraints.

In [26]:
#Disjunctive LE >= 50: LE−𝑀*yLE≤50 and LE+𝑀(1−𝑦LE)≥50

model.constLE501 = pe.Constraint(expr = model.L['E'] - big_M*model.yLE50 <= 50)
model.constLE502 = pe.Constraint(expr = model.L['E'] + big_M*(1 - model.yLE50) >= 50)

#Disjunctive NE <=75: NE+𝑀*yNE≥75 and NE−𝑀(1−𝑦NE)≤75
model.constNE751 = pe.Constraint(expr = model.N['E'] + big_M*model.yNE75 >= 75)
model.constNE752 = pe.Constraint(expr = model.N['E'] - big_M*(1 - model.yNE75) <= 75)

#Disjunctive Linking yLE + yNE >= 1
model.constLink = pe.Constraint(expr = model.yLE50 + model.yNE75 >= 1)

In [27]:
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

ok optimal


In [28]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

optimal objective value minimum cost = $116850.00


In [29]:
DV_solution = pd.DataFrame()
for index in DV_indexes:
    DV_solution.loc['N', index] = model.N[index].value
    DV_solution.loc['A', index] = model.A[index].value
    DV_solution.loc['C', index] = model.C[index].value
    DV_solution.loc['L', index] = model.L[index].value
DV_solution

,E,S,M,W
N,75.0,15.0,60.0,0.0
A,25.0,125.0,0.0,0.0
C,0.0,0.0,0.0,0.0
L,0.0,10.0,50.0,90.0


In [30]:
for index in coef.index:
    print(f'y{index}:', model.y[index].value)

yN: 1.0
yA: 1.0
yC: 0.0
yL: 1.0


We should check that the linking variables seem to be working.
* The `y` variables check usage for each set of City paths. Here we see `y` is 0 because we did not use the Chicago paths.
* The `yLE50` checks if LE >= 50. Above we see LE final count is 0 which means it was not met which matches the y of 0
* the `yNE75` checks if NE <= 75. Above we see NE final count is 75 which means it did meet which matches the y of 1.

In [33]:
print(f'yLE50:', model.yLE50.value)
print(f'yNE75:', model.yNE75.value)

yLE50: 0.0
yNE75: 1.0


In [35]:
for index in coef.index:
    print(f'y{index}:', model.y[index].value)

yN: 1.0
yA: 1.0
yC: 0.0
yL: 1.0


In [36]:
for con in model.component_objects(pe.Constraint):
    print(con.lower, con.slack(), con.upper)

100.0 0.0 None
150.0 0.0 None
110.0 0.0 None
90.0 0.0 None
None 0.0 0.0
None 0.0 0.0
None 0.0 0.0
None 0.0 0.0
None 0.0 0.0
None 90.0 0.0
None 50.0 50.0
50.0 100.0 None
75.0 150.0 None
None 0.0 75.0
1.0 0.0 None
